<a href="https://colab.research.google.com/github/Cousar/Colab-codes-for-TE-project/blob/main/TEcalculatorfromEvaluationDosePlan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#TE exports as txt and Numpy files
#for use you need to adapt the numbers in Line 20 for naming and line 31 for finding the correct evaluation dose to score the TE numbers
import math
import os
import numpy as np
from connect import get_current

def export_track_ends_sums():
    """
    Calculates the total track ends sum for the whole plan and for the
    '3,3cmHalf_Aperture' ROI, and exports these metrics to a text file.

    The output file will contain three metrics:
    1. Maximum Track Ends (Diagnostic)
    2. Sum of Track Ends (Whole Plan)
    3. Sum of Track Ends (Target ROI)
    """
    # --- Data Retrieval ---
    patient = get_current('Patient')
    name =patient.Cases[1].TreatmentPlans[38].Name

    # --- Configuration ---
    TARGET_ROI_NAME = "3,3cmHalf_Aperture"
    OUTPUT_DIR = r'S:\NHC\Scripts\TrackEndsResults\110MeV'
    OUTPUT_FILE_NAME = f'{name}.txt'
    FULL_OUTPUT_PATH = os.path.join(OUTPUT_DIR, OUTPUT_FILE_NAME)

    # Access the Evaluation object (index [24] as used in your original code)
    try:
        # NOTE: If this index [24] is incorrect, this line will fail.
        eval_object = patient.Cases[1].TreatmentDelivery.FractionEvaluations[0].DoseOnExaminations[0].DoseEvaluations[51]
    except IndexError:
        print("Error: DoseEvaluation index [24] is out of range. Check if the evaluation plan exists.")
        return

    # 1. Retrieve and process track ends data for the whole plan
    trackends_eval = eval_object.DoseValues.DoseData
    trackends_eval_flatten = trackends_eval.flatten()

    # Calculate Total Track Ends Sum (Metric 2)
    sum_trackends_eval_flatten = np.sum(trackends_eval_flatten)

    # Calculate Maximum Track Ends (Metric 1 - Diagnostic)
    max_trackends_eval = np.amax(trackends_eval_flatten)

    # 2. Retrieve track ends for the target ROI
    print(f"Retrieving track ends for ROI: {TARGET_ROI_NAME}")

    try:
        # Get voxel indices of the ROI
        dose_grid_indices_roi = patient.Cases[1].TreatmentPlans[0].BeamSets[0].FractionDose.GetDoseGridRoi(RoiName='3,3cmHalf_Aperture').RoiVolumeDistribution.VoxelIndices
        #dose_grid_indices_roi = patient.Cases[1].TreatmentPlans[0].BeamSets[0].FractionDose.GetDoseGridRoi(RoiName='WaterFB').RoiVolumeDistribution.VoxelIndices
    except Exception:
        print(f"Error: The target ROI '{TARGET_ROI_NAME}' was not found or lacks geometry.")
        return

    # Extract track ends distribution in the ROI (1D list)
    trackends_roi = [trackends_eval_flatten[vi] for vi in dose_grid_indices_roi]

    # Calculate Track Ends Sum within the ROI (Metric 3)
    sum_trackends_roi = np.sum(trackends_roi)


    # --- File Output and Export ---

    # Format the results into a list of (Description, Value) tuples
    results = [
        ("Max_Track_Ends_Plan:", max_trackends_eval),
        ("Sum_Track_Ends_Plan:", sum_trackends_eval_flatten),
        (f"Sum_Track_Ends_{TARGET_ROI_NAME}:", sum_trackends_roi)
    ]

    # 1. Ensure the network directory exists
    try:
        os.makedirs(OUTPUT_DIR, exist_ok=True)
    except Exception as e:
        print(f"--- FAILED TO ACCESS DIRECTORY: {OUTPUT_DIR}. Check network access/permissions. Error: {e}")
        return
# --- NEW: Save NumPy Arrays (3D and 1D) ---
    #NPY_FILE_3D = os.path.join(OUTPUT_DIR, 'trackends_eval_3D.npy')
    #NPY_FILE_FLAT = os.path.join(OUTPUT_DIR, 'trackends_eval_flattened.npy')
    # 1. For the 3D file
    NPY_FILE_3D = os.path.join(OUTPUT_DIR, f'{name}_3D.npy')

    # 2. For the flattened file
    NPY_FILE_FLAT = os.path.join(OUTPUT_DIR, f'{name}_flattened.npy')

    try:
        np.save(NPY_FILE_3D, trackends_eval)
        np.save(NPY_FILE_FLAT, trackends_eval_flatten)
        print(f"--- INFO: Saved 3D array to: {NPY_FILE_3D}")
        print(f"--- INFO: Saved 1D array to: {NPY_FILE_FLAT}")
    except Exception as e:
        # We don't return here because we still want to try writing the .txt file.
        print(f"--- FAILED TO WRITE NPY FILES. Check file permissions or path validity. Error: {e}")
    # 2. Write the formatted results to the text file
    try:
        with open(FULL_OUTPUT_PATH, 'w') as output_file:
            for description, value in results:
                # Write the description and the value on a single line
                output_file.write(f"{description}\t{value}\n")
    except Exception as e:
        print(f"--- FAILED TO WRITE FILE: {FULL_OUTPUT_PATH}. Check file permissions or path validity. Error: {e}")
        return

    print(f"*** FINAL CONFIRMATION: Track ends sums successfully exported to: {FULL_OUTPUT_PATH}")

if __name__ == "__main__":
    export_track_ends_sums()